## Lecture 01


### How to train our model of text-data??
1. Load Data
2. Prepocessing data of raw text 
3. Build vocaburary of data
4. Words scoring methods
5. Build model architacture
6. Evaluate Metric

Text data를 사용한 방식의 학습 방식은 크게 이루어져 있습니다. 어떤 기법을 쓰느냐의 문제를 제외하면 이 틀은 크게 벗어나지 않습니다. 따라서 우리는 BoW 방식의 모델링 과정을 순서별로 알아보겠습니다.

#### Load Data

가장 우선적으로 사용할 데이터를 불어와야 합니다. NLP에서 모델의 성능을 평가하는 Benchmark data set의 일부는 다음에 나열된 데이터들이 있습니다.

* MPQA
* **Movie Review**
* IMDB
* CR
* 20News
* TREC-QA

우리는 이번 시간에 Movie Review 데이터를 사용해서 어떻게 Preprecessing 하며 모델에 맞게 transform시키는지 알아보겠습니다.<br>
우선 [Movie Review dataset URL](http://www.cs.cornell.edu/people/pabo/movie-review-data/) 에서 데이터를 다운을 받습니다. 
여기에선 **polarity dataset v2.0** 버젼의 데이터를 사용합니다. 자신의 작업환경에서 압축을 풀면 Pos/Neg 두개의 파일이 있을겁니다.
이제 Pandas 를 사용해서 데이터를 사용하기 용이하게끔 바꾸어 줍니다. 제 경우에는 폴더명을 MR로 바꾸고 실행했습니다.

In [38]:
import pandas as pd
import numpy as np
import pickle
import datetime
import os
import re

In [2]:
label_list = os.listdir("./MR/")
raw_text = []
for label in label_list:
    label_text = []
    txt_data = os.listdir("./MR/{}".format(label))
    for data in txt_data:
        f = open("./MR/{}/{}".format(label, data))
        label_text.append(f.read())
        f.close()
    raw_text.append(label_text)    

In [3]:
num_neg = len(os.listdir("./MR/neg/"))
num_pos = len(os.listdir("./MR/pos/"))

neg_label = np.zeros([num_neg], dtype=np.float32)
pos_label = np.ones([num_pos], dtype=np.float32)
label = np.concatenate([neg_label, pos_label])
label = label.astype(np.float32)

여기까지 살펴보면 각각의 txt 파일의 데이터들을 불러와 저장하고 neg:0 , pos: 1 로 labeling 합니다. 이 데이터들을 항상 이런식으로 불러와서 사용한다면 너무나 비효율적인 방식입니다. 따라서 **Pickle** or **NPZ** or **TF-record**로 저장하고 효율적으로 load 하는 습관을 들여야 합니다. 이 장에서는 pickle을 사용하여 dump하겠습니다.

In [4]:
data_list = [raw_text, label, num_neg, num_pos]
pickle.dump(data_list, open("./MR/raw_dataset.p", "wb"))

**data_list** 에는 neg/pos 로 이루어진 raw text와 0, 1로 매핑된 label, 각각의 label개수를 리스트에 담고 해당경로에 저장합니다. <br>
만들어진 pickle파일을 load하는 방법은 아래와 같습니다.

In [5]:
load = pickle.load(open("./MR/raw_dataset.p", "rb"))
raw_text, label, num_neg, num_pos = load[0], load[1], load[2], load[3]

#### Prepocessing data of raw text 
이제 우리는 raw data를 prepocessing 하는 과정을 거쳐야 합니다. 어떻게 보면 이 과정은 모델을 설정하는 것 만큼이나 매우 중요한 과정이며, 전처리를 어떻게 하는지에 따라서 매우 다른 결과를 가지게 됩니다. 우선 우리가 사용할 모델이 BoW 방식을 이용하기 때문에 BoW 를 알아보고 그에 맞는 전처리를 해보겠습니다. <br>

##### Bag-of-Words
간단한 예를 들어보겠습니다. 
데이터셋을 **["I am a student",  "I am a researcher of Teamlab",  "Teamlab is data science lab"]** 라고 하겠습니다. <br>
단어사전은 단어들의 집합(Vocaburary)으로 이루어집니다. <br>
해당 단어에 index를 부여한 dict 타입이라면 다음과 같이 나타낼 수 있습니다. 당연히 중복은 없애야 겠죠?<br>

**{"I", "am", "a", "student", "researcher", "of", "Teamlab", "is", "data", "science", "lab"}**

BoW 모델을 적용시킬 때 Scoring Word Methods 는 한가지에 국한되지 않습니다. 크게 3가지로 나눠보자면 다음과 같습니다.
* Binary counting
* Counting
* Frequency

3가지 경우에 대해서 ["I am a student"] 라는 데이터가 어떻게 표현되는지 알아보겠습니다.
1. Binary Counting
       Binary Counting 의 경우 데이터가 vocaburary 안에 존재하는지 안하는지만을 따집니다. 따라서 데이터의 표현은 [1,1,1,1,0,0,0,0,0,0,0] 와 같이 나타낼 수 있습니다.
       
2. Counting
        Counting 의 경우 해당 단어들의 등장 횟수를 표현합니다. 따라서 데이터의 표현은 [1,1,1,1,0,0,0,0,0,0,0]입니다. 이 경우에는 모든 단어들이 한번씩밖에 등장하지 않았지만 만약 "I like you but you hate me" 라한다면 you에 해당하는 index 에는 값이 2가 되겠죠?
        
3. Frequency        
       단어들의 빈도수에 의한 계산 방식입니다. TF-IDF를 주로 사용하게 되는데 TF-IDF는 해당 단어가 가지는 중요도를 파악할 수 있습니다. TF-IDF는 아주 많이 사용되어지는 방식이며 좀 더 예를 들어 설명하겠습니다.
       
##### TF-IDF (Term Frequency - Inverse Document Frequency)

수식부터 보게되면 <br>

$W_{i,j} = tf_{i,j} \times \log\frac{N}{df_i} $
로 표현할 수 있습니다. i 는 단어의 id값, j는 문서의 id값, N은 문서의 개수입니다. 이제 앞서 설정한 데이터셋에서 "Teamlab is data science lab"의 "Teamlab"단어에 대한 TF-IDF값을 구해보겠습니다. <br>

$tf_{i,j}$는 해당 문서에서 단어가 몇번 나온지를 나타내는 값입니다. 저희의 경우에는 1이 되겠죠. IDF 는 문서내에서 이 해당 단어가 얼마나 나왔는지를 표현하는 것입니다. 따라서 $\log(3/2)$ 가 됩니다. 최종적으로 계산하면 $1\times \log(3/2)$가 됩니다. <br>

IDF 값을 통해서 우리는 문서를 표현하는 하는 단어의 중요도를 나타낼 수 있습니다. 왜냐하면 is, are 과 같은 단어는 모든 문서에서 많이 등장하게 되어 IDF의 $df_i$가 커지게 되며 결과적으로 TF-IDF값이 낮아지기 때문입니다.

##### Bag-of-Words

이 3가지 scoring methods를 통해서 해당문서를 numeric 하게 표현하게 됩니다. 이번 시간에는 가장 간단하게 Binary counting을 사용합니다. 
사용하기에 앞서 좀 더 살펴볼 점이 있습니다. <br>

Bag-of-Word의 표현은 $[N \times V]$ 로 표현된다는 점입니다. $N$은 데이터의 개수, $V$는 단어사전의 단어 개수 입니다. 이 부분은 꼭 짚고 넘어가야 할 부분입니다. <br>
간단하게 생각을 해보자면 우리가 예로든 데이터셋은 굉장히 작은 데이터일 뿐이지만 실제 학습시켜야하는 데이터의 $V$의 크기는 상당히 클 수 밖에 없습니다.
따라서 연산량이 커지게 됩니다. 또한 차원의 저주에 빠질 가능성이 커지게 됩니다. 따라서 우리는 이러한 문제를 인지하면서 전처리를 해야합니다.<br>
그렇다면 어떻게 $V$를 효율적으로 줄일 수 있을까요.<br>
1. 대/소문자 통일: Teamlab -> teamlab <br>
<br>
2. **stop-words** 제외: Except stop-words(ex: i, me, my, myself) <br>
<br>
3. **stemming**(축약형) / **lemmatization**(품사정보가 남아있는 원형)<br>
<br>
4. 불필요한 특수문자 어휘 제거: ex : %, ### <br>

위의 4가지 방법을 통해 전처리를 하게된다면 sparse한 data를 어느정도 효율적으로 처리할 수 있을겁니다. 하지만 위의 4가지 방법을 **항상 사용해야 좋은것은 아닙니다**. 어떤 데이터이냐 또는 어떠한 문제를 풀고자 하는지에 따라서 유동적인 전처리를 해야합니다. <br>
1, 4의 경우 정규식을 통해서 처리할 수 있습니다.

In [6]:
def clean_str(s):
    # only include alphanumerics
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    
    # insert spaces in words with apostrophes
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    
    # insert spaces in special characters
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\(", " \( ", s)
    s = re.sub(r"\)", " \) ", s)
    s = re.sub(r"\?", " \? ", s)
    
    # only include alphanumerics again
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    
    # reduce multiple spaces to single spaces
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip().lower()

In [7]:
raw_text = np.reshape(raw_text, [-1])
raw_text = raw_text.tolist()
clean_text = [clean_str(x) for x in raw_text]

그럼 정규식을 통해서 text cleaning을 한 결과를 비교해 볼까요? <br>
Raw : i'm not sure who the genius is who came up with the idea of comparing " disturbing behavior " with " scream . " <br>
**Clean** : i'm not sure who the genius is who came up with the idea of comparing disturbing behavior with scream <br>

이제 우리는 **stop-words** 를 제외시키고 **stemming**을 사용하여 축약형 단어로 변환시키겠습니다. 이과정은 nltk 패키지를 사용하면 쉽게 변환할 수 있습니다.

In [8]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
stopword_set = set(stopwords.words('english'))

def stopword_stem(sentence):
    split_sentence = sentence.split()
    stem_sentence = []
    for word in split_sentence:
        stem_word = stemmer.stem(word)
        stem_sentence.append(stem_word)
    filtered_sentence = [w for w in stem_sentence if not w in stopword_set]  
    return " ".join(filtered_sentence)

clean_text = [stopword_stem(x) for x in clean_text]

#### Build vocaburary of data
이제 문서에 존재하는 단어들의 사전을 만듭니다.

In [9]:
vocab = []
for text in clean_text:
    split_text = text.split()
    vocab.extend(split_text)
vocab = set(vocab)    
vocab_size = len(vocab)
print("vocab size is %d"%vocab_size)

vocab size is 25611


#### Words scoring methods
Binary Counting method를 사용합니다. vocab size 가 25345이기 때문에 문서의 크기는 $[N, 25345]$가 됩니다.

In [10]:
x_data = np.zeros([np.shape(clean_text)[0], vocab_size], dtype=np.float32)
for i, text in enumerate(clean_text):
    for j, vocab_word in enumerate(vocab):
        if vocab_word in text:
            x_data[i][j] = 1.0       
            
y_data = np.reshape(label, [len(x_data), 1])
y_data = y_data.astype(np.float32)            

#### Build model architacture
이제 모델을 구현해야 합니다. 이번시간에는 기본적인 Neural Networks를 이용하겠습니다. 

![image](http://www.reckonanalytics.com/wp-content/uploads/2018/02/anns.png)

각각의 레이어의 Feed Forward 계산은 다음과 같이 이루어집니다. $h_j = \max(0, \sum_i u_i w_{ij})$ $u_i$는 이전 레이어의 출력값, $w_{ij}$는 가중치입니다. <br>
우리의 목적은 데이터를 이용해서 긍정과 부정을 예측하는 모델을 구현하는 것이며 우선적으로 데이터를 shuffle 시킵니다. 

In [11]:
randidx = np.random.randint(len(x_data),size=len(x_data))
x_data = x_data[randidx,:]
y_data = y_data[randidx,:]

이제 Train data 와 Test data로 나누어줍니다. 데이터의 개수가 적기 때문에 Val data는 이번 시간에는 제외시키지만 모델의 성능을 좀 더 명확하게 알아보기 위해서는 Val data도 포함되어야 합니다.

In [62]:
split_val = int(len(x_data)*0.9)
train_x = x_data[:split_val]
test_x = x_data[split_val:]

train_y = y_data[:split_val]
test_y = y_data[split_val:]

#### Build model using Tensorflow
지금부터는 텐서플로우를 이용해서 모델을 구현하겠습니다. 텐서플로우에 대한 기본이 없다는 가정하에 설명합니다. <br>
초반에 NN을 구현하는 것이 힘든 이유중 하나는 연산의 과정에서 shape이 헷갈리기 때문입니다. 하나씩 차근차근 알아보겠습니다. <br>
모델의 인풋 아웃풋을 지정하기위해 placeholder를 사용합니다. placeholder란 일종의 틀이라고 보시면 될 것 같습니다. 만약에 placeholder를 쓰지않고 x_data, y_data로 지정한다면 해당하는 shape의 구조만을 이용할 수 있습니다. 하지만 placeholder라는 틀을 만들어 놓고 그 안에 데이터를 담는다면 유동적으로 사용할 수 있습니다. placehoder의 지정은 다음과 같습니다.

In [12]:
import tensorflow as tf
input_x = tf.placeholder(tf.float32, [None, vocab_size])
input_y = tf.placeholder(tf.float32, [None, 1])

Training 시키기 위해 파라메터를 설정합니다. 

In [78]:
batch_size = 128
learning_rate = 0.001
num_data = len(x_data)
decay_lr = 0.9
total_epoch = 10
step_size = int(num_data / batch_size)

이제 히든레이어 $h_j$를 만들기 위해 weight variable를 지정해주고 메트릭스 곱 연산을 해줍니다. 모델의 복잡성에 따라서 혹은 데이터의 복잡성에 따라서 intitializer를 설정할 필요가 있습니다. 여기서는 보편적으로 사용하는 **정규분포**와 **zero initializer**를 사용합니다.

In [14]:
weight_j = tf.get_variable("weight1", shape=[vocab_size, 512], initializer=tf.random_normal_initializer())
bias_j = tf.get_variable("bias1", shape=[512,], initializer=tf.zeros_initializer())

In [15]:
h_j = tf.nn.relu(tf.matmul(input_x, weight_j) + bias_j)

아웃풋 레이어는 binary classification이기 때문에 출력 레이어의 shape은 $[512, 1]$로 지정합니다.

In [16]:
weight_k = tf.get_variable("weight3", shape=[512, 1], initializer=tf.random_normal_initializer())
bias_k = tf.get_variable("bias3", shape=[1], initializer=tf.zeros_initializer())
logits = tf.matmul(h_j, weight_k)+bias_k

모델의 구현은 이것으로 끝입니다. 우리는 이모델을 역전파를 통해서 trainable 변수들을 update시켜야 하기 때문에 
loss function의 error rate를 최소화하는 optimizer가 필요합니다.<Br>
**global optimum**으로 수렴하기 위해선 learning rate를 일정한 주기마다 decay시켜서 gradient를 update시키는 방식을 사용합니다.<br>
* step size : 얼마만큼의 간격으로 learning rate를 decay할 것인가를 정해줍니다.
* decay : decay rate of learning rate
* apply_gradient : gradient 정책을 원하는 방식으로 바꾸고 싶을 때 사용합니다.

In [17]:
# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=input_y))

In [29]:
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate, global_step,
                                           step_size, decay_lr)
optimizer = tf.train.AdamOptimizer(learning_rate)
gradients = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(gradients, global_step=global_step)

#### Accuracy
모델의 성능은 Accuracy로 측정하겠습니다. 
* tf.greater : 이진분류이기 때문에 0.5이상의 값을 가지면 1.0으로 리턴합니다.
* tf.equal : element와의 비교를 통해서 boolean 타입으로 리턴합니다.

In [31]:
predicted_class = tf.greater(logits,0.5)
correct = tf.equal(predicted_class, tf.equal(input_y, 1.0))
accuracy = tf.reduce_mean( tf.cast(correct, 'float') )

이제 모델을 학습시키는 일만 남았습니다. 모델을 학습시킬 때 **시간**을 측정하는 습관을 가지면 좋습니다.

In [81]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(num_data/batch_size)
    start_time = datetime.datetime.now()
    for epoch in range(total_epoch):
        avg_loss = avg_acc = index = 0
        for step in range(total_batch):
            if (index + 1) * batch_size < num_data:
                loss_val, _, acc = sess.run([loss, train_op, accuracy], feed_dict=
                                       {input_x:train_x[index*batch_size:(index+1)*batch_size],
                                        input_y:train_y[index*batch_size:(index+1)*batch_size]})       
                index += 1
            else:
                loss_val, _, acc = sess.run([loss, train_op, accuracy], feed_dict=
                                       {input_x:train_x[index*batch_size:],
                                        input_y:train_y[index*batch_size:]})
                index = 0
                
            avg_loss += loss_val / total_batch
            avg_acc += acc / total_batch
        print("{} epoch ----> loss : {} train acc : {}".format(epoch, avg_loss, acc))    
    print("Training time : {}".format(datetime.datetime.now()-start_time))
    print("Test accuracy : {}".format(sess.run(accuracy, feed_dict={input_x:test_x, input_y:test_y})))

0 epoch ----> loss : 121.5496602376302 train acc : 0.375
1 epoch ----> loss : 58.38635495503743 train acc : 0.625
2 epoch ----> loss : 31.409788513183592 train acc : 1.0
3 epoch ----> loss : 18.36222235361735 train acc : 1.0
4 epoch ----> loss : 10.820548240090844 train acc : 1.0
5 epoch ----> loss : 6.70812530517578 train acc : 1.0
6 epoch ----> loss : 3.875429906447728 train acc : 1.0
7 epoch ----> loss : 2.2635934988666664 train acc : 1.0
8 epoch ----> loss : 1.4497762696210454 train acc : 1.0
9 epoch ----> loss : 1.0304437576118288 train acc : 1.0
Training time : 0:00:00.719320
Test accuracy : 0.8600000143051147


### Finish
이번시간에는 BoW를 활용한 모델을 구현해 보았습니다. BoW 모델이 가지는 한계는 다음과 같습니다.
* 문맥을 전혀 고려하지 않음
* 단어들의 의미론적 표현이 불가능
* 데이터가 방대해질 경우, $V$의 크기가 클수록 연산량이 비효율적임 

따라서 다음시간에는 단어들의 의미론적 표현이 가능한 **Word2Vec**에 대해서 알아보겠습니다.